In [1]:
%%writefile ".env"

PROJECT_ID="feux_cq"

COMMUN_PATH = "C:/"
PATH_INFOCENTRE_APP = ${COMMUN_PATH}Informatique/SIG/Application/Jupyterhub/
PATH_ETUDE = ${COMMUN_PATH}Informatique/SIG/Donnees/Oeil/Traitement_Donnees/SURFACES_BRULEES_SENTINEL/2022/220502_PreparationFeu2020/
PROJECT_PATH =${COMMUN_PATH}Users/oriane.bruyere/Documents/Projet_feux/Script/Step1_detection_error/
DATA_CATALOG_DIR = ${PATH_INFOCENTRE_APP}projets/catalogFiles/ 
DATA_OUTPUT_DIR = ${PROJECT_PATH}output/
SIG_DATA_PATH = ${COMMUN_PATH}Informatique/SIG/Donnees/
DB_USER="jfnguyenvansoc"
DB_PWD="oeil"
DB_HOST="172.20.12.13"
DB_PORT=5432

DB_WORKSPACE="oeil_traitement"
DB_REF="oeil_reference"
DB_EXT="data_externe"
DB_SCHEMA = "feux_cq"
DB_SCHEMA_REF = "feux"

Overwriting .env


## Get missing burned areas files from INSIGHT server

In [2]:
import logging
from dotenv import load_dotenv
from filecmp import dircmp
from os import listdir
import os
import filecmp 
import geopandas as gpd
import pandas as pd
import fiona
from sqlalchemy import create_engine
load_dotenv()
import subprocess
from tobler.util import h3fy
import h3
from shapely.geometry import MultiPolygon
from datetime import datetime

## Open gpkg and add data to POSTGIS

In [3]:
DATA_TEMP ='A:/FEUX_INSIGHT/2024/'

date_start=datetime(2024,1,1)
date_end=datetime(2024,3,31)

In [4]:
fichiers_geopackages = [f for f in os.listdir(DATA_TEMP) if f.endswith('.gpkg')]
gdf_list = []

for fichier_geopackage in fichiers_geopackages:
    chemin_geopackage = os.path.join(DATA_TEMP, fichier_geopackage)
    couches = fiona.listlayers(chemin_geopackage)

    for couche in couches:
        gdf_couches = gpd.read_file(chemin_geopackage, layer=couche, driver='GPKG')
        
        if 'date_' in gdf_couches.columns:
            gdf_couches['date_']=pd.to_datetime(gdf_couches['date_'])
            gdf_couches['date_']=gdf_couches['date_'].dt.date
            gdf_couches['datetexte'] = gdf_couches['date_texte'].apply(lambda x: x[:-11] if len(x) > 8 else x)

        elif 'date_texte' or 'date' in gdf_couches.columns:
            gdf_couches.rename(columns={'date_texte': 'date_'}, inplace=True)
            gdf_couches.rename(columns={'date': 'date_'}, inplace=True)
            
            gdf_couches['datetexte'] = gdf_couches['date_'].apply(lambda x: x[:-11] if len(x) > 8 else x)
            gdf_couches['date_']=pd.to_datetime(gdf_couches['datetexte'])
            gdf_couches['date_']=gdf_couches['date_'].dt.date

    
        gdf_couches['nom'] = gdf_couches['nom'].apply(lambda x: x[:19] + x[30:] if len(x) > 30 else x)
        gdf_list.append(gdf_couches)

gdf = pd.concat(gdf_list, ignore_index=True, sort=False)
gdf=gdf.reset_index(drop=True)

### filtre sur le data set
gdf['date_']=pd.to_datetime(gdf['date_'])
gdf=gdf.loc[(gdf['date_'] >= date_start) & (gdf['date_'] <= date_end)]
gdf=gdf.reset_index(drop=True)

print("Données fusionnées à partir des GeoPackages :")

Données fusionnées à partir des GeoPackages :


### Convertir les surfaces detectees en type MULTIPOLYGONE

In [5]:
# Vérifier si la géométrie est de type Polygon
if gdf['geometry'].geom_type[0] == 'Polygon':
    gdf['geometry'] = gdf['geometry'].apply(lambda poly: MultiPolygon([poly]))

    print(gdf)
else:
    print("La géométrie n'est pas de type Polygon.")

          date_                             nom           province    commune  \
0    2024-01-03  SENTINEL2A_20240103_L2A_T58KFC  Province des Iles      OUVEA   
1    2024-01-03  SENTINEL2A_20240103_L2A_T58KFC  Province des Iles      OUVEA   
2    2024-01-03  SENTINEL2A_20240103_L2A_T58KEC      Province Nord      TOUHO   
3    2024-01-03  SENTINEL2A_20240103_L2A_T58KEC      Province Nord      TOUHO   
4    2024-01-03  SENTINEL2A_20240103_L2A_T58KEC      Province Nord      TOUHO   
...         ...                             ...                ...        ...   
5590 2024-03-31  SENTINEL2B_20240331_L2A_T58KDB      Province Nord  POUEMBOUT   
5591 2024-03-31  SENTINEL2B_20240331_L2A_T58KDB      Province Nord  POUEMBOUT   
5592 2024-03-31  SENTINEL2B_20240331_L2A_T58KDB      Province Nord  POUEMBOUT   
5593 2024-03-31  SENTINEL2B_20240331_L2A_T58KDB      Province Nord  POUEMBOUT   
5594 2024-03-31  SENTINEL2B_20240331_L2A_T58KDB      Province Nord  POUEMBOUT   

       surface       x     

### Ajouter une identifiant H3 et créer un identifiant unique

In [6]:
H3_res = 15
def geo_to_h3(row):
  return h3.geo_to_h3(row.lat,row.lon,resolution = H3_res)

In [7]:
gdf=gdf.to_crs(4326) ## change coordinates to WGS84
gdf=gdf.set_crs("EPSG:4326")

gdf['centroides']=gdf['geometry'].centroid
gdf['lon'] = gdf.centroides.x
gdf['lat'] = gdf.centroides.y

gdf['h3_id'] = gdf.apply(geo_to_h3,axis=1)

gdf['surface_id_h3']=0
for i in range(len(gdf)):
  gdf['surface_id_h3'][i]=str(gdf['nom'][i][-10:])+"_"+str(gdf['datetexte'][i])+"_"+str(gdf['h3_id'][i])

C:\Users\oriane.bruyere\AppData\Local\Temp\ipykernel_24232\1044648674.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroides']=gdf['geometry'].centroid
C:\Users\oriane.bruyere\AppData\Local\Temp\ipykernel_24232\1044648674.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and e

## Création de la base de donnée dans POSTGIS

In [8]:
gdf = gpd.GeoDataFrame(gdf.copy(), crs=gdf.crs, geometry=gdf['geometry'])
gdf=gdf.to_crs(3163) ## change coordinates to RGNC
gdf=gdf.set_crs("EPSG:3163")
gdf=gdf[["nom", "province", "commune", "surface", "geometry","date_","surface_id_h3"]]

In [9]:
from sqlalchemy import text
table = "surfaces_brulees_brute_control"

conex = create_engine(f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PWD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_WORKSPACE")}')
gdf.to_postgis(table, conex,schema=os.getenv("DB_SCHEMA"),if_exists='append')